This will be used for capstone project

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
urldata=requests.get(url).text
soup=BeautifulSoup(urldata,'html.parser')

In [5]:
table = soup.find('table')
#table   

In [6]:
column_names = ['Postal Code', 'Borough', 'Neighbourhood'] 
# instantiate the dataframe
Toronto_df = pd.DataFrame(columns=column_names)

for tr_cell in table.find_all('tr'): 
    int=1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    for td_cell in tr_cell.find_all('td'):
        if int == 1: 
            Postcode_var = str(td_cell.text).strip()
        if int == 2: 
            Borough_var = str(td_cell.text).strip()
            tag_a_Borough = td_cell.find('a')
        if int == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
        
        int +=1
    
    #exclude row with a Not assigned Borough or Postcode
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned'):            
        continue
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):        
        continue     
    
    Toronto_df=Toronto_df.append({'Postal Code': Postcode_var,
                                          'Borough': Borough_var,
                                          'Neighbourhood': Neighbourhood_var
                                          }, ignore_index=True)


In [7]:
Toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
print('The dataframe has unique {} Postal Code and {} number of Rows.'.format(
        len(Toronto_df['Postal Code'].unique()),
        Toronto_df.shape[0]
    ) 
)


The dataframe has unique 103 Postal Code and 103 number of Rows.


In [9]:
GeoCoordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
GeoCoordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#Merging GeoCoordinates_df with Toronto DataFrame

In [10]:
TorontoNew_df=Toronto_df.merge(GeoCoordinates_df)

In [11]:
TorontoNew_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [21]:
Borough_Toronto_df=TorontoNew_df[TorontoNew_df['Borough'].str.contains('Toronto')]
print ('Shape of Borough dataframe is {}'.format(Borough_Toronto_df.shape))
Borough_Toronto_df.head()


Shape of Borough dataframe is (39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [23]:
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 
neighborhoods = Borough_Toronto_df[column_names]
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,Downtown Toronto,St. James Town,43.651494,-79.375418
19,East Toronto,The Beaches,43.676357,-79.293031


In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [28]:
from geopy.geocoders import Nominatim 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [34]:
import folium
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
#map_Toronto.save('Neighborhood_Toronto.html')
map_Toronto